In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, applications
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np
import os

# Paths
train_dir = r"D:\Choudhury_Major\rvf10k\train"
valid_dir = r"D:\Choudhury_Major\rvf10k\valid"

# Parameters
img_size = (224, 224)
batch_size = 32
autotune = tf.data.AUTOTUNE

# Function to process images
def process_image(file_path, label):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, img_size)
    img = tf.image.random_flip_left_right(img)
    img = tf.image.random_brightness(img, max_delta=0.2)
    img = tf.image.random_contrast(img, 0.8, 1.2)
    img = tf.image.random_saturation(img, 0.8, 1.2)
    img = img / 255.0
    return img, label

# Function to prepare dataset
def prepare_dataset(directory):
    class_names = ['real', 'fake']
    all_files, all_labels = [], []
    for idx, name in enumerate(class_names):
        class_path = os.path.join(directory, name)
        for fname in os.listdir(class_path):
            all_files.append(os.path.join(class_path, fname))
            all_labels.append(idx)
    files = tf.constant(all_files)
    labels = tf.constant(all_labels)
    ds = tf.data.Dataset.from_tensor_slices((files, labels))
    ds = ds.map(process_image, num_parallel_calls=autotune)
    ds = ds.shuffle(1000).batch(batch_size).prefetch(autotune)
    return ds, labels

train_ds, train_labels = prepare_dataset(train_dir)
val_ds, val_labels = prepare_dataset(valid_dir)

# Compute class weights
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                  classes=np.unique(train_labels.numpy()),
                                                  y=train_labels.numpy())
class_weights = {i : class_weights[i] for i in range(len(class_weights))}

# Load EfficientNetB4
base_model = applications.EfficientNetB4(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
base_model.trainable = False

# Custom classifier with dropout and batchnorm
inputs = layers.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = models.Model(inputs, outputs)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy', tf.keras.metrics.AUC(name='auc'),
                       tf.keras.metrics.Precision(name='precision'),
                       tf.keras.metrics.Recall(name='recall')])

# Callbacks
early_stop = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-7)

# Train
history = model.fit(train_ds, validation_data=val_ds, epochs=20,
                    callbacks=[early_stop, lr_scheduler], class_weight=class_weights)

# Report training accuracy and loss
avg_accuracy = np.mean(history.history['accuracy'])
avg_loss = np.mean(history.history['loss'])
print(f"Average Training Accuracy: {avg_accuracy:.4f}")
print(f"Average Training Loss: {avg_loss:.4f}")

# Fine-tune
base_model.trainable = True
for layer in base_model.layers[:50]:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy', tf.keras.metrics.AUC(name='auc'),
                       tf.keras.metrics.Precision(name='precision'),
                       tf.keras.metrics.Recall(name='recall')])

finetune_history = model.fit(train_ds, validation_data=val_ds, epochs=10,
                             callbacks=[early_stop, lr_scheduler], class_weight=class_weights)

# Report fine-tuning accuracy and loss
finetune_avg_accuracy = np.mean(finetune_history.history['accuracy'])
finetune_avg_loss = np.mean(finetune_history.history['loss'])
print(f"Average Fine-tuning Accuracy: {finetune_avg_accuracy:.4f}")
print(f"Average Fine-tuning Loss: {finetune_avg_loss:.4f}")

# Evaluation
val_labels = np.concatenate([y for x, y in val_ds], axis=0)
val_preds = (model.predict(val_ds) > 0.5).astype("int32")

precision = precision_score(val_labels, val_preds)
recall = recall_score(val_labels, val_preds)
f1 = f1_score(val_labels, val_preds)

print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}")

# Save the model
model.save("efficientnetb4_deepfake_model.keras")


71686520/71686520 ━━━━━━━━━━━━━━━━━━━━ 15s 0us/step
Epoch 1/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 266s 1s/step - accuracy: 0.4899 - auc: 0.3204 - loss: 0.8717 - precision: 0.1678 - recall: 0.3185 - val_accuracy: 0.5000 - val_auc: 0.4861 - val_loss: 0.7429 - val_precision: 0.5000 - val_recall: 1.0000 - learning_rate: 1.0000e-04
Epoch 2/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 267s 1s/step - accuracy: 0.4993 - auc: 0.3222 - loss: 0.8289 - precision: 0.1736 - recall: 0.3180 - val_accuracy: 0.5000 - val_auc: 0.5049 - val_loss: 0.7550 - val_precision: 0.5000 - val_recall: 1.0000 - learning_rate: 1.0000e-04
Epoch 3/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 240s 1s/step - accuracy: 0.5104 - auc: 0.3135 - loss: 0.8022 - precision: 0.1701 - recall: 0.3107 - val_accuracy: 0.5000 - val_auc: 0.5393 - val_loss: 0.7417 - val_precision: 0.5000 - val_recall: 1.0000 - learning_rate: 1.0000e-04
Epoch 4/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 243s 1s/step - accuracy: 0.5027 - auc: 0.3121 - loss: 0.7964 - precision: 0.1705 - recall: 0.30

In [1]:
import cv2

def crop_face(image_np):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    image_np = (image_np * 255).astype(np.uint8)
    gray = cv2.cvtColor(image_np, cv2.COLOR_RGB2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4)

    if len(faces) > 0:
        x, y, w, h = faces[0]
        face = image_np[y:y+h, x:x+w]
        face = cv2.resize(face, (224, 224))
        return face / 255.0
    else:
        return cv2.resize(image_np, (224, 224)) / 255.0


ModuleNotFoundError: No module named 'cv2'

In [2]:
def process_image(file_path, label):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (256, 256))
    img = tf.numpy_function(crop_face, [img], tf.float32)
    return img, label


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, applications
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
import numpy as np
import os
import cv2

# Paths
data_dir = r"D:\Choudhury_Major\rvf10k"

# Parameters
img_size = (224, 224)
batch_size = 32
autotune = tf.data.AUTOTUNE

# Face cropping function
def crop_face(image_np):
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    image_np = (image_np * 255).astype(np.uint8)
    gray = cv2.cvtColor(image_np, cv2.COLOR_RGB2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=4)
    if len(faces) > 0:
        x, y, w, h = faces[0]
        face = image_np[y:y+h, x:x+w]
        face = cv2.resize(face, img_size)
        return face / 255.0
    else:
        return cv2.resize(image_np, img_size) / 255.0

# Process image
def process_image(file_path, label):
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (256, 256))
    img = tf.numpy_function(crop_face, [img], tf.float32)
    return img, label

# Prepare dataset
file_paths, labels = [], []
class_names = ['real', 'fake']
for idx, name in enumerate(class_names):
    class_path = os.path.join(data_dir, name)
    for fname in os.listdir(class_path):
        file_paths.append(os.path.join(class_path, fname))
        labels.append(idx)

train_files, val_files, train_labels_np, val_labels_np = train_test_split(file_paths, labels, test_size=0.2, stratify=labels, random_state=42)

train_ds = tf.data.Dataset.from_tensor_slices((train_files, train_labels_np))
train_ds = train_ds.map(process_image, num_parallel_calls=autotune).shuffle(1000).batch(batch_size).prefetch(autotune)

val_ds = tf.data.Dataset.from_tensor_slices((val_files, val_labels_np))
val_ds = val_ds.map(process_image, num_parallel_calls=autotune).batch(batch_size).prefetch(autotune)

# Compute class weights
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                  classes=np.unique(train_labels_np),
                                                  y=train_labels_np)
class_weights = {i: class_weights[i] for i in range(len(class_weights))}

# Load EfficientNetB4
base_model = applications.EfficientNetB4(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
base_model.trainable = False

# Custom classifier with dropout and batchnorm
inputs = layers.Input(shape=(224, 224, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(1, activation='sigmoid')(x)
model = models.Model(inputs, outputs)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy', tf.keras.metrics.AUC(name='auc'),
                       tf.keras.metrics.Precision(name='precision'),
                       tf.keras.metrics.Recall(name='recall')])

# Callbacks
early_stop = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-7)

# Train
history = model.fit(train_ds, validation_data=val_ds, epochs=20,
                    callbacks=[early_stop, lr_scheduler], class_weight=class_weights)

# Report training accuracy and loss
avg_accuracy = np.mean(history.history['accuracy'])
avg_loss = np.mean(history.history['loss'])
print(f"Average Training Accuracy: {avg_accuracy:.4f}")
print(f"Average Training Loss: {avg_loss:.4f}")

# Fine-tune
base_model.trainable = True
for layer in base_model.layers[:50]:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='binary_crossentropy',
              metrics=['accuracy', tf.keras.metrics.AUC(name='auc'),
                       tf.keras.metrics.Precision(name='precision'),
                       tf.keras.metrics.Recall(name='recall')])

finetune_history = model.fit(train_ds, validation_data=val_ds, epochs=10,
                             callbacks=[early_stop, lr_scheduler], class_weight=class_weights)

# Report fine-tuning accuracy and loss
finetune_avg_accuracy = np.mean(finetune_history.history['accuracy'])
finetune_avg_loss = np.mean(finetune_history.history['loss'])
print(f"Average Fine-tuning Accuracy: {finetune_avg_accuracy:.4f}")
print(f"Average Fine-tuning Loss: {finetune_avg_loss:.4f}")

# Evaluation
val_labels_tensor = tf.constant(val_labels_np)
val_preds = (model.predict(val_ds) > 0.5).astype("int32")

precision = precision_score(val_labels_np, val_preds)
recall = recall_score(val_labels_np, val_preds)
f1 = f1_score(val_labels_np, val_preds)

print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1-Score: {f1:.4f}")

# Save the model
model.save("efficientnetb4_deepfake_model.keras")

# Grad-CAM Visual Explanation
import matplotlib.pyplot as plt
import tensorflow.keras.backend as K

def get_img_array(img_path):
    img = tf.io.read_file(img_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (256, 256))
    img = tf.numpy_function(crop_face, [img], tf.float32)
    return tf.expand_dims(img, axis=0)

def make_gradcam_heatmap(img_array, model, last_conv_layer_name):
    grad_model = tf.keras.models.Model([model.inputs], [model.get_layer(last_conv_layer_name).output, model.output])
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        loss = predictions[:, 0]
    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = tf.reduce_sum(tf.multiply(pooled_grads, conv_outputs), axis=-1)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

def display_gradcam(img_path, heatmap, alpha=0.4):
    import matplotlib.cm as cm
    img = cv2.imread(img_path)
    img = cv2.resize(img, img_size)
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cm.jet(heatmap)[:, :, :3]
    superimposed_img = heatmap * alpha + img / 255.0
    plt.imshow(superimposed_img)
    plt.axis('off')
    plt.title("Grad-CAM Heatmap")
    plt.show()

# Example:
# test_img_path = "/mnt/data/rvf10k/rvf10k/all/fake/example.jpg"
# img_array = get_img_array(test_img_path)
# heatmap = make_gradcam_heatmap(img_array, model, "top_conv")  # name depends on EfficientNetB4's layer
# display_gradcam(test_img_path, heatmap)


In [3]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


ModuleNotFoundError: No module named 'cv2'

In [4]:
!pip install opencv-python


  Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_python-4.11.0.86-cp37-abi3-win_amd64.whl (39.5 MB)


In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [2]:
import os
import cv2
import numpy as np

image_data = []
labels = []

# Update this to your correct path
base_path = r"D:\Choudhury_Major\rvf10k"
sub_dirs = ['train', 'valid']
categories = ['real', 'fake']
img_size = 128

for sub_dir in sub_dirs:
    for category in categories:
        folder_path = os.path.join(base_path, sub_dir, category)
        label = categories.index(category)

        if not os.path.exists(folder_path):
            print(f"[ERROR] Folder not found: {folder_path}")
            continue

        for img_file in os.listdir(folder_path):
            if img_file.lower().endswith(('.jpg', '.jpeg', '.png')):
                img_path = os.path.join(folder_path, img_file)
                try:
                    img = cv2.imread(img_path)
                    img = cv2.resize(img, (img_size, img_size))
                    image_data.append(img)
                    labels.append(label)
                except Exception as e:
                    print(f"Error loading image: {img_path}")

# Convert to numpy arrays
image_data = np.array(image_data)
labels = np.array(labels)

print("✅ Dataset Loaded Successfully!")
print("Images shape:", image_data.shape)
print("Labels shape:", labels.shape)


✅ Dataset Loaded Successfully!
Images shape: (10000, 128, 128, 3)
Labels shape: (10000,)


In [7]:
import os
import cv2
import numpy as np

def load_dataset(base_path, img_size=128):
    data = []
    labels = []
    categories = ['real', 'fake']

    for label, category in enumerate(categories):
        folder = os.path.join(base_path, category)
        if not os.path.exists(folder):
            print(f"[ERROR] Folder not found: {folder}")
            continue

        for file in os.listdir(folder):
            if file.endswith(('.jpg', '.jpeg', '.png')):
                img_path = os.path.join(folder, file)
                try:
                    img = cv2.imread(img_path)
                    img = cv2.resize(img, (img_size, img_size))
                    data.append(img)
                    labels.append(label)
                except:
                    pass
    
    return np.array(data), np.array(labels)

# 👇 Update this path according to your system
base_path = "D:/Choudhury_Major/rvf10k"

# Load train data
X_train, y_train = load_dataset(os.path.join(base_path, "train"))

# Load valid data
X_valid, y_valid = load_dataset(os.path.join(base_path, "valid"))

print("✅ Dataset Loaded Successfully!")
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_valid shape:", X_valid.shape)
print("y_valid shape:", y_valid.shape)


✅ Dataset Loaded Successfully!
X_train shape: (7000, 128, 128, 3)
y_train shape: (7000,)
X_valid shape: (3000, 128, 128, 3)
y_valid shape: (3000,)


In [8]:
from sklearn.model_selection import train_test_split

# Suppose X and y are from one combined folder
# For example:
X = np.concatenate([X_train, X_valid])
y = np.concatenate([y_train, y_valid])

# Split into new train and val sets
X_train_new, X_val_new, y_train_new, y_val_new = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("✅ Split done!")
print("X_train_new shape:", X_train_new.shape)
print("X_val_new shape:", X_val_new.shape)


✅ Split done!
X_train_new shape: (8000, 128, 128, 3)
X_val_new shape: (2000, 128, 128, 3)


In [10]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Normalize the pixel values between 0 and 1
X_train = X_train.astype('float32') / 255.0
X_valid = X_valid.astype('float32') / 255.0

# Image Data Augmentation for training set
train_datagen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

# For validation set, no augmentation needed
val_datagen = ImageDataGenerator()

# Flow the data using generators
train_generator = train_datagen.flow(X_train, y_train, batch_size=32)
val_generator = val_datagen.flow(X_valid, y_valid, batch_size=32)

print("Data normalized and generators created.")


Data normalized and generators created.


In [11]:
weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = {i : weights[i] for i in range(len(weights))}


In [12]:
def build_model():
    model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
        MaxPooling2D(2,2),
        BatchNormalization(),
        
        Conv2D(64, (3,3), activation='relu'),
        MaxPooling2D(2,2),
        BatchNormalization(),
        
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.4),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = build_model()


C:\Users\PRIYANSHU\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ModelCheckpoint("best_model.h5", save_best_only=True)
]

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=15,
    class_weight=class_weights,
    callbacks=callbacks
)


Epoch 1/15


C:\Users\PRIYANSHU\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step - accuracy: 0.5251 - loss: 3.1060

219/219 ━━━━━━━━━━━━━━━━━━━━ 30s 130ms/step - accuracy: 0.5251 - loss: 3.0996 - val_accuracy: 0.4987 - val_loss: 1.9954
Epoch 2/15
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step - accuracy: 0.5515 - loss: 0.8102

219/219 ━━━━━━━━━━━━━━━━━━━━ 30s 139ms/step - accuracy: 0.5516 - loss: 0.8102 - val_accuracy: 0.5223 - val_loss: 0.9471
Epoch 3/15
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step - accuracy: 0.5642 - loss: 0.8215

219/219 ━━━━━━━━━━━━━━━━━━━━ 29s 133ms/step - accuracy: 0.5642 - loss: 0.8214 - val_accuracy: 0.5670 - val_loss: 0.7554
Epoch 4/15
219/219 ━━━━━━━━━━━━━━━━━━━━ 31s 140ms/step - accuracy: 0.5654 - loss: 0.7139 - val_accuracy: 0.5390 - val_loss: 0.8024
Epoch 5/15
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step - accuracy: 0.5804 - loss: 0.7247

219/219 ━━━━━━━━━━━━━━━━━━━━ 30s 139ms/step - accuracy: 0.5804 - loss: 0.7247 - val_accuracy: 0.5773 - val_loss: 0.6980
Epoch 6/15
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step - accuracy: 0.5700 - loss: 0.7182

219/219 ━━━━━━━━━━━━━━━━━━━━ 30s 139ms/step - accuracy: 0.5701 - loss: 0.7181 - val_accuracy: 0.5860 - val_loss: 0.6759
Epoch 7/15
219/219 ━━━━━━━━━━━━━━━━━━━━ 31s 140ms/step - accuracy: 0.5854 - loss: 0.6908 - val_accuracy: 0.6083 - val_loss: 0.7006
Epoch 8/15
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 130ms/step - accuracy: 0.5726 - loss: 0.6786

219/219 ━━━━━━━━━━━━━━━━━━━━ 30s 138ms/step - accuracy: 0.5725 - loss: 0.6786 - val_accuracy: 0.5823 - val_loss: 0.6712
Epoch 9/15
219/219 ━━━━━━━━━━━━━━━━━━━━ 0s 126ms/step - accuracy: 0.5707 - loss: 0.6731

219/219 ━━━━━━━━━━━━━━━━━━━━ 30s 135ms/step - accuracy: 0.5708 - loss: 0.6731 - val_accuracy: 0.5997 - val_loss: 0.6620
Epoch 10/15
219/219 ━━━━━━━━━━━━━━━━━━━━ 30s 135ms/step - accuracy: 0.5852 - loss: 0.6642 - val_accuracy: 0.6160 - val_loss: 0.6779
Epoch 11/15
219/219 ━━━━━━━━━━━━━━━━━━━━ 29s 133ms/step - accuracy: 0.5787 - loss: 0.6713 - val_accuracy: 0.5723 - val_loss: 0.6997
Epoch 12/15
219/219 ━━━━━━━━━━━━━━━━━━━━ 29s 133ms/step - accuracy: 0.5915 - loss: 0.6626 - val_accuracy: 0.5693 - val_loss: 0.7192


In [16]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Predict on validation data
y_pred = (model.predict(X_valid) > 0.5).astype("int32")

# Evaluation metrics
print("Accuracy:", accuracy_score(y_valid, y_pred))
print("Precision:", precision_score(y_valid, y_pred))
print("Recall:", recall_score(y_valid, y_pred))
print("F1 Score:", f1_score(y_valid, y_pred))


94/94 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step
Accuracy: 0.5996666666666667
Precision: 0.6027491408934708
Recall: 0.5846666666666667
F1 Score: 0.5935702199661591


In [17]:
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# Load VGG19 base model without the top layer
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
base_model.trainable = False  # Freeze base model layers

# Add custom top layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(64, activation='relu')(x)
output = Dense(1, activation='sigmoid')(x)  # binary classification

# Final model
model = Model(inputs=base_model.input, outputs=output)

# Compile model
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.summary()


80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 17s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)           │ (None, 128, 128, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 128, 128, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 128, 128, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 64, 64, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 64, 64, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 64, 64, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 32, 32, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 32, 32, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 32, 32, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 32, 32, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv4 (Conv2D)                │ (None, 32, 32, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 16, 16, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 16, 16, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 16, 16, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 16, 16, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv4 (Conv2D)                │ (None, 16, 16, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 8, 8, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 8, 8, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 8, 8, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 8, 8, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv4 (Conv2D)                │ (None, 8, 8, 512)           │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼──────────────

 Total params: 20,057,281 (76.51 MB)

 Trainable params: 32,897 (128.50 KB)

 Non-trainable params: 20,024,384 (76.39 MB)

In [18]:
# Early stopping to prevent overfitting
early_stop = EarlyStopping(patience=5, restore_best_weights=True)

# Train
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=20,
    callbacks=[early_stop]
)


Epoch 1/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 364s 2s/step - accuracy: 0.5122 - loss: 0.7134 - val_accuracy: 0.6197 - val_loss: 0.6673
Epoch 2/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 2339s 11s/step - accuracy: 0.5461 - loss: 0.6906 - val_accuracy: 0.6383 - val_loss: 0.6526
Epoch 3/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 344s 2s/step - accuracy: 0.5741 - loss: 0.6726 - val_accuracy: 0.6387 - val_loss: 0.6440
Epoch 4/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 1861s 9s/step - accuracy: 0.5924 - loss: 0.6642 - val_accuracy: 0.6463 - val_loss: 0.6359
Epoch 5/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 424s 2s/step - accuracy: 0.6108 - loss: 0.6560 - val_accuracy: 0.6537 - val_loss: 0.6305
Epoch 6/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 449s 2s/step - accuracy: 0.6133 - loss: 0.6534 - val_accuracy: 0.6600 - val_loss: 0.6285
Epoch 7/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 438s 2s/step - accuracy: 0.6303 - loss: 0.6485 - val_accuracy: 0.6630 - val_loss: 0.6223
Epoch 8/20
219/219 ━━━━━━━━━━━━━━━━━━━━ 437s 2s/step - accuracy: 0.6434 - loss: 0.6435 - val_a

In [24]:
from sklearn.model_selection import train_test_split

# Assuming X and y are your image data and labels
X_train, X_val, y_train, y_val = train_test_split(
    image_data, labels, test_size=0.2, stratify=labels, random_state=42
)


In [25]:
val_generator = val_datagen.flow(X_val, y_val, batch_size=32, shuffle=False)


In [26]:
y_pred = (model.predict(val_generator) > 0.5).astype("int32")
y_true = y_val

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print("Accuracy:", accuracy_score(y_true, y_pred))
print("Precision:", precision_score(y_true, y_pred))
print("Recall:", recall_score(y_true, y_pred))
print("F1 Score:", f1_score(y_true, y_pred))


C:\Users\PRIYANSHU\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


63/63 ━━━━━━━━━━━━━━━━━━━━ 33s 517ms/step
Accuracy: 0.6275
Precision: 0.599687255668491
Recall: 0.767
F1 Score: 0.6731022378236069
